### 本项目需要使用Thchs30和ST-CMDS两个数据集, 首先需要下载数据集并将其解压至`dataset/`目录中
Thchs30数据集下载地址:http://cn-mirror.openslr.org/resources/18/data_thchs30.tgz

St-CMDS数据集下载地址:http://cn-mirror.openslr.org/resources/38/ST-CMDS-20170001_1-OS.tar.gz

`dataset`中目录结构要求如下
```
dataset/
├── data_thchs30
├── st-cmds
├── ST-CMDS-20170001_1-OS
└── thchs30
```
需要的python依赖包如下
* python_speech_features
* TensorFlow (1.13+)
* Keras (2.3+)
* Numpy
* wave
* matplotlib
* math
* Scipy
* h5py

In [1]:
import platform as plat
import os

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


from SpeechModel251 import ModelSpeech
from readdata24 import DataSpeech

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#进行配置，使用95%的GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.95
#config.gpu_options.allow_growth=True   #不全部占满显存, 按需分配
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)

datapath = 'dataset/'
modelpath = 'model_speech/'
epochs = 4
batch_size = 4
save_step = 10
AUDIO_LENGTH = 1600

if(not os.path.exists(modelpath)):
    os.makedirs(modelpath)

这里仅仅做演示用, 仅使用20条语音数据进行训练, 您可以将`datalist`目录下的文件拷贝至`dataset`目录下进行完整数据集的训练

In [3]:

ms = ModelSpeech(datapath)
data = DataSpeech(datapath, "train")

num_data = data.GetDataNum()  # 获取数据的数量

yielddatas = data.data_genetator(batch_size, AUDIO_LENGTH)
for epoch in range(epochs):  # 迭代轮数
    print('[running] train epoch %d .' % epoch)
    n_step = 0 # 迭代数据数
    
    for cur_iter in range(num_data//save_step):
        print("[message] epoch %d . Have train datas %d+" % (epoch, n_step * save_step))

        ms._model.fit_generator(yielddatas, save_step)
        n_step += 1       

        ms.SaveModel(comment="_e_" + str(epoch) + "_step_" + str(n_step * save_step))

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 1600, 200, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 1600, 200, 32 288         the_input[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1600, 200, 32 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 1600, 200, 32 9248        dropout_1[0][0]                  
____________________________________________________________________________________________

10/10 [==============================] - 53s 5s/step - loss: 81.6873
[running] train epoch 2 .
[message] epoch 2 . Have train datas 0+
Epoch 1/1
10/10 [==============================] - 51s 5s/step - loss: 125.7560
[message] epoch 2 . Have train datas 10+
Epoch 1/1
10/10 [==============================] - 48s 5s/step - loss: 94.8878
[running] train epoch 3 .
[message] epoch 3 . Have train datas 0+
Epoch 1/1


StopIteration: 

生成语言模型所需的词频文件, 这里导入的是ST-CMDS数据集的2字词, 如果您需要训练自己的语言模型, 可以按照如下方式使用您的语料库生成词频文件

In [63]:
# -*- coding: utf-8 -*-
def sub_run(path,n):    #  n 记录每次切片的一组中包含的字符数
    files = os.listdir(path)
    tmp_str={}
    for file in files:
        if not file.endswith(".txt"):
            continue
        file_name = os.path.join(path, file)
        with open(file_name, 'rb') as f:
            stxt = str(f.read(), "utf-8")
            for i in list(range(len(stxt)-1)):
                tmp_str[stxt[i:i+n]]= tmp_str.get(stxt[i:i+n], 0) + 1          
                
    tmp_str=sorted(tmp_str.items(),key=lambda d:d[1],reverse = True)
    dict_name = "languageModel2.txt" 
    with open(dict_name, 'w') as f:
        for tmp in tmp_str:
            f.writelines(" ".join([tmp[0], str(tmp[1])]))
            f.write("\n")
filepath="dataset/ST-CMDS-20170001_1-OS/"    #用于统计用的文本路径
sub_run(filepath,2)